In [ ]:
import numpy as np
import torch

import pytorch_model_summary

import utils.functions as fns
from models.transformer import MultiHeadAttentionLayer
from models.transformer import get_transformer_encoder

## Window Operations

In [ ]:
# 4-class cyclic shifting
split_heads = torch.arange(72).view(2, 6, 6, 1)
split_heads = split_heads.expand(-1, -1, -1, 8)
split_heads = split_heads.view(2, 6, 6, 4, 2).permute(0, 3, 1, 2, 4).contiguous()

shifted_heads = fns.cyclic_shift(split_heads, 1)
# shifted_heads = shifted_heads.permute(0, 2, 3, 1, 4).contiguous().view(2, 6, 6, -1)
print(shifted_heads)

In [ ]:
# window partitioning
# shifted_heads = shifted_heads.view(2, 6, 6, 4, 2).permute(0, 3, 1, 2, 4).contiguous()
partitions = fns.partition_window(shifted_heads, 2)
print(partitions)

In [ ]:
# window merging
merged = fns.merge_window(partitions, 2)
print(merged)

In [ ]:
# Make masking matrix.
mask = fns.masking_matrix(4, 6, 6, 2, 1)
print(mask)

In [ ]:
# Masking
attn_values = torch.matmul(partitions, partitions.transpose(-1, -2))
attn_values.masked_fill_(mask, -1)
print(attn_values)

In [ ]:
# Make masking matrix when query != key.
mask = fns.masking_matrix(4, 8, 8, 4, 2,
                             4, 4, 2, 1)
print(mask[0, 1], '\n')
print(mask[0, 2], '\n')
print(mask[0, 3], '\n')

## 2D Relative Position Bias (for Windows)

In [ ]:
# Example window index
window_size = 3
coord_index = np.arange(window_size*window_size).reshape((window_size, window_size))
print(coord_index)

In [ ]:
# Coordinate indices along each axis
axis_size = window_size * 2 - 1
coord_x = np.repeat(np.arange(window_size) * axis_size, window_size)
coord_y = np.tile(np.arange(window_size), window_size)
print(coord_x)
print(coord_y)

In [ ]:
# Relative coordinate indices along each axis
relative_x = coord_x[:, np.newaxis] - coord_x
relative_y = coord_y[:, np.newaxis] - coord_y
print(relative_x)
print(relative_y)

In [ ]:
# Relative coordinate indices in 2D window
relative_coord = relative_x + relative_y
relative_coord += relative_coord[-1, 0]
print(relative_coord)

In [ ]:
# Defined function
print(fns.relative_position_index(2).reshape((4, 4)))  # 2x2 window
print(fns.relative_position_index(3).reshape((9, 9)))  # 3x3 window

In [ ]:
# Example window index when key != query
query_window_size = 4
key_window_size = 2
qk_ratio = query_window_size // key_window_size

In [ ]:
# Coordinate indices along each axis
axis_size = query_window_size * 2 - qk_ratio

query_coord_x = np.repeat(np.arange(query_window_size) * axis_size, query_window_size)
query_coord_y = np.tile(np.arange(query_window_size), query_window_size)
print(query_coord_x)
print(query_coord_y)

key_coord_x = np.repeat(np.arange(key_window_size) * axis_size * qk_ratio, key_window_size)
key_coord_y = np.tile(np.arange(key_window_size) * qk_ratio, key_window_size)
print(key_coord_x)
print(key_coord_y)

In [ ]:
# Relative coordinate indices along each axis
relative_x = query_coord_x[:, np.newaxis] - key_coord_x
relative_y = query_coord_y[:, np.newaxis] - key_coord_y
print(relative_x)
print(relative_y)

In [ ]:
# Relative coordinate indices in 2D window
relative_coord = relative_x + relative_y
relative_coord -= relative_coord[0, -1]
print(relative_coord)

In [ ]:
# Defined function
print(fns.relative_position_index(2).reshape((4, 4)))  # 2x2 window
print(fns.relative_position_index(3).reshape((9, 9)))  # 3x3 window

In [ ]:
print(fns.relative_position_index(6, 2).reshape((36, 4)))  # 6x6 window - 2x2 window
print()
print(fns.relative_position_index(6, 3).reshape((36, 9)))  # 6x6 window - 3x3 window

## Multi-head Attention Layer

In [ ]:
msa_module = MultiHeadAttentionLayer(128, 4, 28, 28, 4, True)
print(pytorch_model_summary.summary(msa_module, torch.zeros(16, 28, 28, 128)))

In [ ]:
modules = fns.clone_layer(msa_module, 3)
model = torch.nn.Sequential(*modules)

print(pytorch_model_summary.summary(model, torch.zeros(16, 28, 28, 128)))

## Transformer Encoder

In [ ]:
encoder = get_transformer_encoder(d_embed=128,
                                positional_encoding=None,
                                relative_position_embedding=True,
                                n_layer=12,
                                n_head=4,
                                d_ff=128*4,
                                n_patch=28,
                                window_size=4)
print(pytorch_model_summary.summary(encoder, torch.zeros(16, 28, 28, 128)))